In [1]:
import sys
import numpy as np
import parsl
from parsl import python_app
from parsl.config import Config
from parsl.executors.threads import ThreadPoolExecutor
from transformers import EarlyStoppingCallback
import json
import pandas as pd
import os
import re
import pandas as pd
from transformers import BertModel, BertTokenizer
import torch
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter

/home/lyncoln/Git/similaridade_workflow/venv_linux/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Configuração básica do Parsl com ThreadPoolExecutor
config = Config(executors=[ThreadPoolExecutor()])
# Start Parsl on a single computer
parsl.load(config)

In [4]:
#Constroi a lista com os caminhos dos workflows
os.chdir(os.path.join(os.pardir,os.pardir, 'workflows_galaxy'))
nomes_arquivos = []
diretorio = os.getcwd()

for item in os.listdir(diretorio):
    caminho_completo = os.path.join(diretorio, item)
    if os.path.isfile(caminho_completo):
        nomes_arquivos.append(item)

#nomes_arquivos = nomes_arquivos[0:10]

print(len(nomes_arquivos))

1014


In [5]:
dic_workflows = {}
@python_app
def processar_arquivo(arquivo):
    # Carregar o arquivo JSON
    with open(arquivo, 'r') as file:
        fileName = file.name
        dados = file.read()
        dic_workflows[fileName] = dados

        return dic_workflows[fileName]


# Inicia o processamento dos arquivos em paralelo
futures = [processar_arquivo(arquivo) for arquivo in nomes_arquivos]

resultados = [future.result() for future in futures]

In [ ]:
# texto_completo = " ".join(dic_workflows.keys()) + " " + " ".join(dic_workflows.values())

# # Separar o texto em palavras
# palavras = texto_completo.split()

# # Contar a frequência de cada palavra
# frequencias = Counter(palavras)

# # Criar um DataFrame a partir do dicionário de frequências
# df = pd.DataFrame(list(frequencias.items()), columns=['Palavra', 'Frequência'])

# # Ordenar o DataFrame pela frequência das palavras em ordem decrescente
# df = df.sort_values(by='Frequência', ascending=False)

# # Exibir o DataFrame
# print(df[0:50])

In [7]:
df = pd.DataFrame(list(dic_workflows.items()), columns=['name', 'text'])
print(df)

                       name                                               text
0     0c86c39dcd9e08c6.json  {\n    "a_galaxy_workflow": "true",\n    "anno...
1     3680984663c813e1.json  {\n    "a_galaxy_workflow": "true",\n    "anno...
2     692a2b0bb818336d.json  {\n    "a_galaxy_workflow": "true",\n    "anno...
3     49f8b32c3206f76c.json  {\n    "a_galaxy_workflow": "true",\n    "anno...
4     25adc55d2a26e34b.json  {\n    "a_galaxy_workflow": "true",\n    "anno...
...                     ...                                                ...
1009  d5cba8a5ba6880fd.json  {\n    "a_galaxy_workflow": "true",\n    "anno...
1010  e0da87cadb1e6d5f.json  {\n    "a_galaxy_workflow": "true",\n    "anno...
1011  fbf75fbb72b488bd.json  {\n    "a_galaxy_workflow": "true",\n    "anno...
1012  0d8eb75d28fa7df2.json  {\n    "a_galaxy_workflow": "true",\n    "anno...
1013  f8238234db6f04c3.json  {\n    "a_galaxy_workflow": "true",\n    "anno...

[1014 rows x 2 columns]


In [8]:
# Carregar o modelo e o tokenizer
tokenizer = BertTokenizer.from_pretrained('google-bert/bert-large-uncased')
model = BertModel.from_pretrained('google-bert/bert-large-uncased')

texts = df['text'].tolist()

# Função para converter texto em embedding
def text_to_embedding(text):
    encoded_input = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        model_output = model(**encoded_input)
    # Pegar a média dos embeddings de todos os tokens para representar o texto
    return model_output.last_hidden_state.mean(dim=1).squeeze().numpy()

# Converter todos os textos para embeddings
embeddings = [text_to_embedding(text) for text in texts]

# Converter a lista de embeddings em um array 2D
embeddings_array = np.vstack(embeddings)

# Calcular a matriz de similaridade
similarity_matrix = cosine_similarity(embeddings_array)

# Identificar o índice do texto mais similar para cada texto
#similar_indices = similarity_matrix.argsort(axis=1)[:, -2]  # -2 porque o mais similar é o próprio texto

# Identificar os índices dos três textos mais similares para cada texto
similar_indices = similarity_matrix.argsort(axis=1)[:, :-4:-1]  # -4 porque os três mais similares excluem o próprio texto

# Criar coluna no DataFrame para os índices dos textos mais similares
df['top3_descrition'] = similar_indices.tolist()

# Criar coluna no DataFrame para os textos mais similares
#df['top3_similar_text_descrition'] = [[df.loc[idx, 'text'] for idx in indices] for indices in similar_indices]


#df['similar_index_descrition'] = similar_indices
#df['similar_text_descrition'] = df.loc[similar_indices, 'text'].values

/home/lyncoln/Git/similaridade_workflow/venv_linux/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
# Carregar o modelo e o tokenizer
tokenizer = BertTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
model = BertModel.from_pretrained('allenai/scibert_scivocab_uncased')

texts = df['text'].tolist()

# Função para converter texto em embedding
def text_to_embedding(text):
    encoded_input = tokenizer(text, return_tensors='pt', max_length=512, padding=True, truncation=True)
    with torch.no_grad():
        model_output = model(**encoded_input)
    # Pegar a média dos embeddings de todos os tokens para representar o texto
    return model_output.last_hidden_state.mean(dim=1).squeeze().numpy()

# Converter todos os textos para embeddings
embeddings = [text_to_embedding(text) for text in texts]

# Converter a lista de embeddings em um array 2D
embeddings_array = np.vstack(embeddings)

# Calcular a matriz de similaridade
similarity_matrix = cosine_similarity(embeddings_array)

# Identificar os índices dos três textos mais similares para cada texto
similar_indices = similarity_matrix.argsort(axis=1)[:, :-4:-1]  # -4 porque os três mais similares excluem o próprio texto

# Criar coluna no DataFrame para os índices dos textos mais similares
df['top3_descrition_scibert'] = similar_indices.tolist()

# Criar coluna no DataFrame para os textos mais similares
#df['top3_similar_text_descrition_scibert'] = [[df.loc[idx, 'text'] for idx in indices] for indices in similar_indices]

/home/lyncoln/Git/similaridade_workflow/venv_linux/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [12]:
os.chdir(os.path.join(os.pardir,"/scr/similaridade")
df.to_csv("top3_json.csv")

In [16]:
os.chdir(os.path.join(os.pardir,"/scr/similaridade")
os.getcwd()

'/home/lyncoln/Git/similaridade_workflow'